In [18]:
# Import dependencies
import subprocess
import json
import os
import web3
import bit
from web3.middleware import geth_poa_middleware
from dotenv import load_dotenv
from web3.gas_strategies.time_based import medium_gas_price_strategy

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv('mnemonic')

# Import constants.py and necessary functions from bit and web3
from constants import *

In [30]:
w3=web3.Web3(web3.Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.set_gas_price_strategy(medium_gas_price_strategy)


In [31]:
# Create a function called `derive_wallets`
def derive_wallets(coin):
    command = ['./derive','--mnemonic=%s'%mnemonic, '--format=json', '--numderive=3','-g','--coin=%s'%coin]
    p = subprocess.Popen(command, stdout=subprocess.PIPE)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [32]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {BTCTEST:derive_wallets(BTCTEST),ETH:derive_wallets(ETH)}

In [33]:
coins

{'btc-test': [{'path': "m/44'/1'/0'/0/0",
   'address': 'mgoM8UQg4x4xqFKkggSHeCdL5PJ3VDaLYy',
   'xprv': 'tprv8kWVs8gJEegZpqNK5HC1KDBBaTNKmAYJVbiRYadxCmMKEkcDPkjWc5EtMi2hK3z6oWuWzUMg2g31ea1n9Kq82JqeRrL88gtg66wvNFr28YS',
   'xpub': 'tpubDHCY1YiYP2NEiJQ6xvrbicqJ9UtFvVjD4uKCq6gFd39i5Erz29Z6nZrkXqacPULTGvqGV9FemxfyjpZUKv1nPf2xB2qoeAePWNmb3BosW2Q',
   'privkey': 'cPHEAg4rDAWw6z5kFAdv4t2EMobZTEvxbUiQ5DHXPxVgpNUomupP',
   'pubkey': '028233763afe31c606859e6c9e08e9f05f5ebe40c51f5cc6880e4322aad3fb6102',
   'pubkeyhash': '0e11d7bc50516dfe59dc9a0ec7a152781af2015b',
   'index': 0},
  {'path': "m/44'/1'/0'/0/1",
   'address': 'mqMtUBqmZfw8orA6tsuPjyfFvLBT6HzvBk',
   'xprv': 'tprv8kWVs8gJEegZryENjr3C7pRkYvRiLap1bD1waauNb8SYq1DRpBgKNpryyq9CPmikJzAkZ4KEzP6CcGveLp1Nu3zxdMv2P8GnG38Ua7Ug7iP',
   'xpub': 'tpubDHCY1YiYP2NEkSGAdVhnXE5s7wweVuzvAWcis6wg1QEwfVUCSaVuZKUr9wmyHxxwR8hBvhPB5Kudg4qQHCshpYgp9FD9qFGbnjBzJxrnMEX',
   'privkey': 'cRyiMzgVFYGsUwjEJ9YVnLCSCfpH669pU2AkhEMHXTgGJ39Kn8b1',
   'pubkey': '025a0a

In [64]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == BTCTEST:
        return bit.PrivateKeyTestnet(priv_key)
    if coin == ETH:
        return web3.eth.Account.privateKeyToAccount(priv_key)

In [65]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == BTCTEST:
        return bit.PrivateKeyTestnet.prepare_transaction(account, [(to, amount, BTC)])
    if coin == ETH:
        am = w3.toWei(amount, "ether")
        gas_est = w3.eth.estimateGas({"from":account, "to":to, "amount":am})
#         w3.middleware_onion.inject(geth_poa_middleware, layer=0)
        #print(w3.isConnected())
        return {'to': to,
                'from': account,
                'value': am,
                'gas': gas_est,
#                 'gasPrice': w3.eth.generateGasPrice(),
                'gasPrice':250000,
                'nonce': w3.eth.getTransactionCount(account),
                'chainId': w3.eth.chain_id
               }

In [66]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):

    
    if coin == BTCTEST:
        rawtx = create_tx(coin, account, to, amount)
        signed = account.signTransaction(rawtx)
        return bit.network.NetworkAPI.broadcast_tx_testnet(signed)
    if coin == ETH:
        rawtx = create_tx(coin, account.address, to, amount)
        signed = account.signTransaction(rawtx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    

In [67]:
#print(priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey']))
#print(priv_key_to_account(ETH, coins[ETH][0]['privkey']))
#privkey=(priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey']))
#print(create_tx(BTCTEST,privkey,'tb1qsgx55dp6gn53tsmyjjv4c2ye403hgxynxs0dnm',0.000001))
#print(send_tx(BTCTEST,privkey,'tb1qsgx55dp6gn53tsmyjjv4c2ye403hgxynxs0dnm',0.000001))
privkey=priv_key_to_account(ETH, coins[ETH][0]['privkey'])
send_tx(ETH,privkey,'0x80464f155168FF5dAE6A41f4977C6794900DA5bb',2)

HexBytes('0x11f4d02be680c48216157c517a0e2235e8fb7aaa5b2efe57c3ef0eb417a8b820')